## Iterators
<hr>


### Manually Consuming an Iterator

**You need to process items in an iterable, but for whatever reason, you can’t or don’t want to use a for loop.**

To manually consume an iterable, use the next() function and write your code to catch the StopIteration exception.

In [1]:
with open('passwd.txt') as f:
    try:
        line = next(f, None)
        print(line, end='')
    except StopIteration:
        pass

asdf


Normally, StopIteration is used to signal the end of iteration. However, if you’re using next() manually (as shown), you can also instruct it to return a terminating value, such as None, instead.

In [2]:
with open('passwd.txt') as f:
    while True:
        line = next(f, None)
        if line is None:
            break
        print(line, end='')

asdf


The basic mechanism of what happens during iteration:

In [3]:
items = [1, 2, 3]

it = iter(items)

print (next(it))

print (next(it))

print (next(it))

print (next(it))

1
2
3


StopIteration: 

### Delegation Iteration

**You have built a custom container object that internally holds a list, tuple, or some other iterable. You would like to make iteration work with your new container.**

Typically, all you need to do is define an `__iter__()` method that delegates iteration to the internally held container.

Python’s iterator protocol requires __iter__() to return a special iterator object that implements a __next__() method to carry out the actual iteration. If all you are doing is iterating over the contents of another container, you don’t really need to worry about the underlying details of how it works. All you need to do is to forward the iteration request along.

The use of the iter() function here is a bit of a shortcut that cleans up the code. iter(s) simply returns the underlying iterator by calling s.__iter__(), much in the same way that len(s) invokes s.__len__().

In [4]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
root = Node(0)
child1 = Node(1)
child2 = Node(2)

root.add_child(child1)
root.add_child(child2)

for i in root:
    print(i)

Node(1)
Node(2)


### Creating New Iteration Patterns with Generators

**You want to implement a custom iteration pattern that’s different than the usual built-in functions (e.g., range(), reversed(), etc.)**

If you want to implement a new kind of iteration pattern, define it using a generator function.

In [5]:
def frange(start, step, increment):
    x = start
    while x < step:
        yield x
        x += increment

To use such a function, you iterate over it using a for loop or use it with some other function that consumes an iterable (e.g., sum(), list(), etc.). 


In [7]:
for i in frange(0, 5, 0.5):
    print(i)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5


The mere presence of the yield statement in a function turns it into a generator. Unlike a normal function, a generator only runs in response to iteration.

The key feature is that a generator function only runs in response to "next" operations carried out in iteration. Once a generator function returns, iteration stops.

### Implementing the Iterator Protocol

**You are building custom objects on which you would like to support iteration, but would like an easy way to implement the iterator protocol.**

By far, the easiest way to implement iteration on an object is to use a generator function. In “Delegating Iteration”, a Node class was presented for representing tree structures. Perhaps you want to implement an iterator that traverses nodes in a depth-first pattern.

In [25]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()

root = Node(0)
child1 = Node(1)
child2 = Node(2)

root.add_child(child1)
root.add_child(child2)
child1.add_child(Node(3))
child1.add_child(Node(4))
child2.add_child(Node(5))

for ch in root.depth_first():
    print (ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


In [12]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield DepthFirstIterator(self)

        
class DepthFirstIterator(object):
    
    def __init__(self, start_node):
        self._node = start_node
        self._children_iter = None
        self._child_iter = None
        
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._children_iter is None:
            self._children_iter = iter(self._node)
            return self._node
        elif self._child_iter:
            try:
                nextchild = next(self._child_iter)
                return nextchild
            except StopIteration:
                self._child_iter = None
                return next(self)
        else:
            self._child_iter = next(self._children_iter).depth_first()
            return next(self)
        


x
